#### Imports

In [66]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.models import load_model

In [2]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [3]:
final_dataset.head()

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,1
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,1,2
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,3
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,4


In [71]:
# columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter', 'age', 'y']
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter','y']
n_features = 4

In [5]:
grouped = final_dataset.groupby('product_id')
display(grouped)

In [6]:
scalers = {col: {} for col in columns}
scaled_data_list = []

for product_id, group in grouped:
    scaled_group = group.copy()
    for col in columns:
        scaler = StandardScaler()
        scaled_group[col] = scaler.fit_transform(group[[col]])
        scalers[col][product_id] = scaler
    scaled_data_list.append(scaled_group)

# Combinar todos los datos escalados en un solo DataFrame
scaled_data = pd.concat(scaled_data_list)

In [8]:
display(scaled_data[columns])

,plan_precios_cuidados,cust_request_qty,cust_request_tn,close_quarter,y
0,0.0,0.461891,-1.704215,-0.707107,-1.576908
1,0.0,-0.046911,-2.039835,-0.707107,-2.042103
2,0.0,0.786658,-0.435961,1.414214,-0.323111
3,0.0,-1.703222,-1.074260,-0.707107,-1.117043
4,0.0,2.865166,0.273781,-0.707107,0.353285
...,...,...,...,...,...
21615,0.0,-0.652507,-0.792584,-0.816497,-0.792584
21616,0.0,-0.268679,-0.649760,1.224745,-0.649760
21617,0.0,0.115148,-0.595870,-0.816497,-0.595870
21618,0.0,-0.012794,-0.290890,-0.816497,-0.290890


#### Funcion para preparar los datos y crear el modelo

El objetivo es predecir 2 dias en el futuro, por lo que la idea es re-armar el dataset. Donde el valor de X sera el conjunto de datos hasta N-2 e Y va a ser N (siendo N la cantidad de ventas para ese producto)

In [102]:
def prepare_data(data,validation=False):
    X, y = [], []
    unique_product_ids = data['product_id'].unique()
    
    max_seq_length = 0
    
    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        max_seq_length = max(max_seq_length, len(product_values))
    
    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        pad_length = max_seq_length - len(product_values)
        
        # Aplicar padding con ceros a la izquierda para igualar la longitud
        padded_product_values = np.pad(product_values[:, :-1], ((pad_length, 0), (0, 0)), mode='constant')
        
        if validation:
            X.append(padded_product_values[2:]) #<= TODO: Cambiar por X.append(padded_product_values[:-2]). Para esto hay que agregar 2 registros con 0s extra en caso
                                                #         de NO validacion, sino no va a matchear el tamaño de la entrada a la red neuronal.
            
            y.append(product_id)
        else:
            X.append(padded_product_values[:-2])  # Todas las observaciones menos las últimas dos
            y.append(product_values[-1, -1])      # Última observación
    
    if validation:
        return np.array(X), np.array(y), max_seq_length
    else:
        return np.array(X), np.array(y), max_seq_length


Definimos la funcion para crear el modelo LSTM, sobre este se ejecutara la optimizacion bayesiana

In [24]:
n_steps = 2  # número de pasos de tiempo
epochs = 100
batch_size = 32
predictions = []

In [52]:
data = scaled_data[columns].values
X, y, max_seq_length = prepare_data(scaled_data)

#### Revisamos que la funcion genere registros coherentes

In [35]:
# Muestro todos los registros del ultimo producto
unique_product_ids = scaled_data['product_id'].unique()
product_data = scaled_data[scaled_data['product_id'] == unique_product_ids[0]] #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft
product_values = product_data[columns].values
display(product_values)

array([[ 0.        ,  0.46189074, -1.70421544, -0.70710678, -1.57690786],
       [ 0.        , -0.04691078, -2.0398351 , -0.70710678, -2.04210334],
       [ 0.        ,  0.78665767, -0.43596114,  1.41421356, -0.32311076],
       [ 0.        , -1.7032221 , -1.07425999, -0.70710678, -1.11704256],
       [ 0.        ,  2.865166  ,  0.27378143, -0.70710678,  0.35328467],
       [ 0.        ,  1.44701708,  0.35490544,  1.41421356,  0.4140519 ],
       [ 0.        , -0.59901455, -1.22422686, -0.70710678, -1.25068433],
       [ 0.        ,  2.23728327, -0.5685186 , -0.70710678, -0.44544444],
       [ 0.        , -0.59901455, -0.35136974,  1.41421356, -0.27688808],
       [ 0.        , -1.76817549, -0.07823625, -0.70710678,  0.14086658],
       [ 0.        ,  0.89491331,  0.39077184, -0.70710678,  0.6195406 ],
       [ 0.        , -0.01443409, -1.18431507,  1.41421356, -1.18702359],
       [ 0.        ,  0.00721704, -0.67712904, -0.70710678, -0.77989182],
       [ 0.        , -1.45423413, -1.0

In [45]:
display(X[0]) #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft

array([[ 0.        ,  0.46189074, -1.70421544, -0.70710678],
       [ 0.        , -0.04691078, -2.0398351 , -0.70710678],
       [ 0.        ,  0.78665767, -0.43596114,  1.41421356],
       [ 0.        , -1.7032221 , -1.07425999, -0.70710678],
       [ 0.        ,  2.865166  ,  0.27378143, -0.70710678],
       [ 0.        ,  1.44701708,  0.35490544,  1.41421356],
       [ 0.        , -0.59901455, -1.22422686, -0.70710678],
       [ 0.        ,  2.23728327, -0.5685186 , -0.70710678],
       [ 0.        , -0.59901455, -0.35136974,  1.41421356],
       [ 0.        , -1.76817549, -0.07823625, -0.70710678],
       [ 0.        ,  0.89491331,  0.39077184, -0.70710678],
       [ 0.        , -0.01443409, -1.18431507,  1.41421356],
       [ 0.        ,  0.00721704, -0.67712904, -0.70710678],
       [ 0.        , -1.45423413, -1.01800077, -0.70710678],
       [ 0.        ,  1.67435393,  1.40999356,  1.41421356],
       [ 0.        , -0.56653786, -0.57996109, -0.70710678],
       [ 0.        ,  0.

In [47]:
display(y[0]) #<= cambiar el indice entre 0 y -1

0.3617453793178112

In [49]:
display(len(X[0]))

34

#### Entrenamiento del modelo

In [57]:
display(max_seq_length)

34

In [72]:
from keras.metrics import MeanAbsoluteError

def create_model(units, learning_rate, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(max_seq_length - n_steps, n_features)))
    model.add(Dropout(0.1))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[MeanAbsoluteError()])
    return model

#### Armado de los modelos

In [73]:
param_space = {
    # 'depth': [2, 3, 4, 5, 6],
    # 'activation': ['relu', 'tanh', 'swish', 'selu'],
    'units': [64, 128, 256, 512],
    # 'dropout_rate': [0.1, 0.2, 0.3],
    'learning_rate': [0.01, 0.001, 0.0001],
    'epochs': [10, 20, 30, 50, 100],
    'batch_size': [32, 64, 128],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    # 'l2_penalty': [0.001, 0.01, 0.1]
}

In [74]:
model = KerasRegressor(
    build_fn=create_model,
    verbose=1,
    units=64,
    learning_rate=0.01
)

# cv = TimeSeriesSplit(n_splits=5).split(X)

bayes_search = BayesSearchCV(
    model,
    param_space,
    n_iter=50,
    # cv=cv,
    verbose=1,
    n_jobs=-1,
    scoring='neg_mean_squared_error'
)

In [75]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

bayes_search.fit(X, y, callbacks = [early_stopping])

5/5 [==============================] - 17s 3s/step - loss: 0.3721 - mean_absolute_error: 0.4634
Epoch 15/20
5/5 [==============================] - 18s 3s/step - loss: 0.4075 - mean_absolute_error: 0.4798
Epoch 14/20
5/5 [==============================] - 16s 3s/step - loss: 0.3801 - mean_absolute_error: 0.4745
Epoch 15/20
5/5 [==============================] - 17s 3s/step - loss: 0.3914 - mean_absolute_error: 0.4702
Epoch 15/20
5/5 [==============================] - 16s 3s/step - loss: 0.3575 - mean_absolute_error: 0.4545
Epoch 16/20
5/5 [==============================] - 17s 3s/step - loss: 0.4004 - mean_absolute_error: 0.4739
Epoch 15/20
5/5 [==============================] - 16s 3s/step - loss: 0.3789 - mean_absolute_error: 0.4666
Epoch 16/20
5/5 [==============================] - 16s 3s/step - loss: 0.3829 - mean_absolute_error: 0.4656
Epoch 16/20
5/5 [==============================] - 16s 3s/step - loss: 0.3696 - mean_absolute_error: 0.4667
Epoch 16/20
5/5 [=======================

2024-06-07 17:23:53.550790: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-07 17:23:53.552511: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-07 17:23:53.584922: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-07 17:23:53.609247: I tensorflow/core/platform/cpu_featu

Epoch 1/30
Epoch 1/30
Epoch 1/30
Epoch 1/30
Epoch 1/30
20/20 [==============================] - 39s 1s/step - loss: 0.8973 - mean_absolute_error: 0.7727
Epoch 2/30
20/20 [==============================] - 39s 2s/step - loss: 0.9365 - mean_absolute_error: 0.7924
Epoch 2/30
20/20 [==============================] - 39s 2s/step - loss: 0.8685 - mean_absolute_error: 0.7648
Epoch 2/30
20/20 [==============================] - 40s 2s/step - loss: 0.9167 - mean_absolute_error: 0.7884
Epoch 2/30
20/20 [==============================] - 40s 2s/step - loss: 0.8747 - mean_absolute_error: 0.7609
Epoch 2/30
20/20 [==============================] - 35s 2s/step - loss: 0.5569 - mean_absolute_error: 0.5831
Epoch 3/30
20/20 [==============================] - 37s 2s/step - loss: 0.5507 - mean_absolute_error: 0.5796
Epoch 3/30
20/20 [==============================] - 37s 2s/step - loss: 0.5341 - mean_absolute_error: 0.5619
Epoch 3/30
20/20 [==============================] - 38s 2s/step - loss: 0.6348 - mea

/Users/mmarchetta/Desktop/LaboratorioIII/myenv/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mmarchetta/Desktop/LaboratorioIII/myenv/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mmarchetta/Desktop/LaboratorioIII/myenv/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mmarchetta/Desktop/LaboratorioIII/myenv/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at w

Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
20/20 [==============================] - 11s 54ms/step - loss: 0.8052 - mean_absolute_error: 0.7168
Epoch 2/20
20/20 [==============================] - 11s 59ms/step - loss: 0.7702 - mean_absolute_error: 0.7027
Epoch 2/20
20/20 [==============================] - 1s 67ms/step - loss: 0.5475 - mean_absolute_error: 0.5568
Epoch 3/20
20/20 [==============================] - 11s 81ms/step - loss: 0.7263 - mean_absolute_error: 0.6653
Epoch 2/20
20/20 [==============================] - 1s 56ms/step - loss: 0.5331 - mean_absolute_error: 0.5550
Epoch 3/20
20/20 [==============================] - 1s 62ms/step - loss: 0.6091 - mean_absolute_error: 0.5846
Epoch 3/20
20/20 [==============================] - 1s 72ms/step - loss: 0.6214 - mean_absolute_error: 0.5981
Epoch 3/20
20/20 [==============================] - 2s 80ms/step - loss: 0.5123 - mean_absolute_error: 0.5537
Epoch 3/20
20/20 [==============================] - 1s 54ms/step - loss: 

...

Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
20/20 [==============================] - 11s 70ms/step - loss: 0.7817 - mean_absolute_error: 0.7109
Epoch 2/20
20/20 [==============================] - 11s 83ms/step - loss: 0.7407 - mean_absolute_error: 0.6792
Epoch 2/20
20/20 [==============================] - 11s 83ms/step - loss: 0.7780 - mean_absolute_error: 0.7030
Epoch 2/20
20/20 [==============================] - 11s 88ms/step - loss: 0.7305 - mean_absolute_error: 0.6834
Epoch 2/20
20/20 [==============================] - 11s 76ms/step - loss: 0.8196 - mean_absolute_error: 0.7174
Epoch 2/20
20/20 [==============================] - 1s 73ms/step - loss: 0.5430 - mean_absolute_error: 0.5580
Epoch 3/20
20/20 [==============================] - 2s 82ms/step - loss: 0.5409 - mean_absolute_error: 0.5508
Epoch 3/20
20/20 [==============================] - 2s 80ms/step - loss: 0.5647 - mean_absolute_error: 0.5849
Epoch 3/20
20/20 [==============================] - 1s 74ms/step - loss

20/20 [==============================] - 2s 82ms/step - loss: 0.2482 - mean_absolute_error: 0.3791
Epoch 20/20
5/5 [==============================] - 2s 45ms/step


/Users/mmarchetta/Desktop/LaboratorioIII/myenv/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
25/25 [==============================] - 11s 79ms/step - loss: 1.0102 - mean_absolute_error: 0.8513
Epoch 2/100
25/25 [==============================] - 2s 81ms/step - loss: 0.7683 - mean_absolute_error: 0.6950
Epoch 3/100
25/25 [==============================] - 2s 91ms/step - loss: 0.6142 - mean_absolute_error: 0.5995
Epoch 4/100
25/25 [==============================] - 2s 90ms/step - loss: 0.5161 - mean_absolute_error: 0.5482
Epoch 5/100
25/25 [==============================] - 2s 98ms/step - loss: 0.4751 - mean_absolute_error: 0.5213
Epoch 6/100
25/25 [==============================] - 3s 116ms/step - loss: 0.4478 - mean_absolute_error: 0.5045
Epoch 7/100
25/25 [==============================] - 3s 114ms/step - loss: 0.4223 - mean_absolute_error: 0.4876
Epoch 8/100
25/25 [==============================] - 4s 178ms/step - loss: 0.4047 - mean_absolute_error: 0.4803
Epoch 9/100
25/25 [==============================] - 3s 135ms/step - loss: 0.3943 - mean_absolute_error: 0.4

BayesSearchCV(estimator=KerasRegressor(build_fn=<function create_model at 0x7fb408966670>, learning_rate=0.01, units=64),
              n_jobs=-1, scoring='neg_mean_squared_error',
              search_spaces={'batch_size': [32, 64, 128],
                             'epochs': [10, 20, 30, 50, 100],
                             'learning_rate': [0.01, 0.001, 0.0001],
                             'optimizer': ['adam', 'rmsprop', 'sgd'],
                             'units': [64, 128, 256, 512]},
              verbose=1)

#### Guardo los mejores hiperparametros

In [80]:
import json

# Obtener los mejores hiperparámetros
best_params = bayes_search.best_params_

# Guardar los mejores hiperparámetros en un archivo JSON
with open('best_params.json', 'w') as file:
    json.dump(best_params, file)

print('Los mejores hiperparámetros han sido guardados en best_params.json.')


Los mejores hiperparámetros han sido guardados en best_params.json.


In [85]:
with open('best_params.json', 'r') as file:
    best_params = json.load(file)

# Reconstruir el modelo con los mejores hiperparámetros
mejor_modelo = create_model(units=best_params['units'], learning_rate=best_params['learning_rate'], optimizer=best_params['optimizer'])
mejor_modelo.fit(X, y)
# Verificar la configuración del nuevo modelo
print("Configuración del nuevo modelo:")
print(mejor_modelo.summary())

25/25 [==============================] - 11s 111ms/step - loss: 1.0302 - mean_absolute_error: 0.8636
Configuración del nuevo modelo:
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 34, 128)           68096     
                                                                 
 dropout_6 (Dropout)         (None, 34, 128)           0         
                                                                 
 lstm_7 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 199809 (780.50 KB)
Trainable params: 19

#### Hago las predicciones

In [ ]:
# best_params = BayesSearchCV.best_params_

# best_units = best_params['build_fn__units']
# best_dropout_rate = best_params['build_fn__dropout_rate']
# best_optimizer = best_params['build_fn__optimizer']

# best_model = create_model(units=best_units, dropout_rate=best_dropout_rate, optimizer=best_optimizer)
# best_model.fit(X, y)

In [103]:
X_f, product_ids, _ = prepare_data(scaled_data, validation=True)

In [104]:
scaled_predictions = mejor_modelo.predict(X_f)
display(scaled_predictions)

25/25 [==============================] - 3s 65ms/step


array([[-0.03749761],
       [ 0.07719515],
       [-0.28504974],
       [-0.16752963],
       [ 0.00191359],
       [-0.46567208],
       [-0.5009722 ],
       [-0.50684524],
       [ 0.08870511],
       [-0.2601701 ],
       [-0.35600978],
       [-0.5058283 ],
       [-0.24785617],
       [-0.13520795],
       [-0.28447428],
       [-0.52350885],
       [-0.49021673],
       [-0.45785466],
       [-0.32442707],
       [-0.38365418],
       [-0.3616345 ],
       [-0.1745336 ],
       [-0.1690569 ],
       [-0.40086722],
       [-0.3435718 ],
       [-0.2885229 ],
       [-0.19864361],
       [-0.32962388],
       [-0.51589715],
       [-0.6266643 ],
       [-0.37555188],
       [-0.04145496],
       [-0.29670253],
       [-0.3770427 ],
       [-0.03380069],
       [-0.28932574],
       [-0.44118848],
       [-0.37785086],
       [-0.06397761],
       [-0.43387252],
       [-0.12032729],
       [-0.36022997],
       [-0.26667163],
       [-0.23500235],
       [ 0.0042713 ],
       [-0

In [111]:
scaled_predictions_1d =  scaled_predictions.reshape(-1)
display(scaled_predictions_1d)

array([-0.03749761,  0.07719515, -0.28504974, -0.16752963,  0.00191359,
       -0.46567208, -0.5009722 , -0.50684524,  0.08870511, -0.2601701 ,
       -0.35600978, -0.5058283 , -0.24785617, -0.13520795, -0.28447428,
       -0.52350885, -0.49021673, -0.45785466, -0.32442707, -0.38365418,
       -0.3616345 , -0.1745336 , -0.1690569 , -0.40086722, -0.3435718 ,
       -0.2885229 , -0.19864361, -0.32962388, -0.51589715, -0.6266643 ,
       -0.37555188, -0.04145496, -0.29670253, -0.3770427 , -0.03380069,
       -0.28932574, -0.44118848, -0.37785086, -0.06397761, -0.43387252,
       -0.12032729, -0.36022997, -0.26667163, -0.23500235,  0.0042713 ,
       -0.3462353 , -0.20838092, -0.49287635, -0.33186266, -0.20829011,
       -0.48015544, -0.34228492, -0.25859642, -0.4107947 , -0.14103614,
       -0.1835302 , -0.4608457 , -0.17010643, -0.20884869, -0.31820124,
       -0.33453554, -0.43422347, -0.18936421, -0.3089106 , -0.11791895,
       -0.4735755 ,  0.13032307, -0.40687776, -0.23368268, -0.26

In [117]:
predictions_df = pd.DataFrame({
    'product_id': product_ids,
    'predicted_y': scaled_predictions_1d
})

# Iterar sobre cada fila del DataFrame predictions_df
for index, row in predictions_df.iterrows():
    # Obtener el ID del producto y la predicción escalada para este producto
    product_id = row['product_id']
    scaled_prediction = row['predicted_y']
    
    # Obtener el escalador correspondiente a 'predicted_y' para este producto
    scaler = scalers['y'][product_id]
    
    # Aplicar la inversa del escalador a la predicción 'predicted_y' para este producto
    inverse_scaled_prediction = scaler.inverse_transform([[scaled_prediction]])[0][0]
    
    # Reemplazar la predicción escalada con la predicción invertida en el DataFrame final
    predictions_df.at[index, 'predicted_y'] = inverse_scaled_prediction

# Guardar el DataFrame final con las predicciones invertidas
predictions_df.to_csv('predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv después de aplicar la inversa de los scalers.')


Todas las predicciones han sido generadas y guardadas en predictions.csv después de aplicar la inversa de los scalers.
